In [ ]:
#| default_exp models.BT000X

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import torch, re, inspect
from dataclasses import dataclass
from typing import Optional, List, Tuple, Mapping, Any
from transformers import BertLMHeadModel, BatchEncoding
from transformers.utils.generic import ModelOutput
from fastcore.meta import *

from xcai.test_utils import *
from xcai.losses import *

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## Setup

In [ ]:
block = Test.from_cfg('train')

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
bsz = 20
batch = block.train.one_batch(bsz)

In [ ]:
batch

{'lbl2data_idx': tensor([ 93458, 194001,   2544,  67030, 207194,  65848, 138083, 190705, 212057,
        212058, 106148,  57415,   5098,  35257, 108559, 230538, 260048,   3056,
        110493, 199088, 139747,  26243, 174656,   2980,  26704,  61452, 270585,
         27086,  45172,  81895,  20557,  56645,  78517,  87964, 193369, 208162]), 'lbl2data_identifier': ['List_of_scientific_misconduct_incidents', "List_of_Russian_Nature_Reserves_(class_1a_'zapovedniks')", 'Tourism_in_Japan', 'List_of_Special_Places_of_Scenic_Beauty,_Special_Historic_Sites_and_Special_Natural_Monuments', 'Public_Order_Act', 'The_House_That_Shadows_Built', 'University_of_Toronto', 'Knowledge_Forum', 'Knowledge_building', 'OISE', 'Skardu', 'Communes_of_the_Morbihan_department', 'File_system', 'WinFS', 'Science_and_technology_in_Flanders', 'Agoria', 'IWT', 'Honorific_nicknames_in_popular_music', 'Music_of_Serbia', 'Serbian_hip_hop', 'Chief_Cashier_of_the_Bank_of_England', 'List_of_Major_League_Baseball_players_named_

In [ ]:
batch.keys()

dict_keys(['lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask'])

## Output

In [ ]:
#| export
@dataclass
class XCModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor, ...]] = None
    attentions: Optional[Tuple[torch.FloatTensor, ...]] = None
    cross_attentions: Optional[Tuple[torch.FloatTensor, ...]] = None
    

## XCModel

In [ ]:
class XCModel(BertLMHeadModel):

    def __init__(self, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        

## BT0001

In [ ]:
#| export
class BT0001(BertLMHeadModel):

    def __init__(self, cfg):
        super().__init__(cfg)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        data_token_type_ids:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_token_type_ids:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o = self.bert(
            data_input_ids,
            data_attention_mask,
            data_token_type_ids,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        data_embeds = data_o[0]
        data_logits = self.cls(data_embeds)

        data_embed = data_embeds.mean(dim=1)
        
        if lbl2data_input_ids is not None and lbl2data_data2ptr is not None:
            lbl2data_o = self.bert(
                lbl2data_input_ids,
                lbl2data_attention_mask,
                lbl2data_token_type_ids,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict
            )
            
            lbl2data_embeds = lbl2data_o[0]
            lbl2data_embed = lbl2data_embeds.mean(dim=1)
            
            return data_logits, lbl2data_input_ids, lbl2data_data2ptr, data_embed, lbl2data_embed

        return data_logits, lbl2data_input_ids, lbl2data_data2ptr
        

In [ ]:
m = BT0001.from_pretrained('bert-base-uncased')

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
out = m(**batch)

In [ ]:
for o in out: print(o.shape)

torch.Size([10, 16, 30522])
torch.Size([28, 10])
torch.Size([10])
torch.Size([10, 768])
torch.Size([28, 768])


## BT0002

In [ ]:
#| export
class BT0002(BertLMHeadModel):

    def __init__(self,
                 config,
                 tn_targ:Optional[int]=None, 
                 ig_tok:Optional[int]=0,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.loss_fn = MultiCrossEntropy(tn_targ=tn_targ, ig_tok=ig_tok, reduce='mean')

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        data_token_type_ids:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_token_type_ids:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o = self.bert(
            data_input_ids,
            data_attention_mask,
            data_token_type_ids,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        data_embeds = data_o[0]
        data_logits = self.cls(data_embeds)
        
        loss = None
        if (lbl2data_input_ids is not None 
            or lbl2data_data2ptr is not None):
            loss = self.loss_fn(data_logits, lbl2data_input_ids, lbl2data_data2ptr)

        if not return_dict:
            o = (data_logits,) + data_o[2:]
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            logits=data_logits,
            hidden_states=data_o.hidden_states,
            attentions=data_o.attentions,
            cross_attentions=data_o.cross_attentions,
        )
        

In [ ]:
m = BT0002.from_pretrained('bert-base-uncased', tn_targ=10_000, ig_tok=0)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BT0002 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(m, batch)

In [ ]:
m, b = m.to('cuda'), b.to('cuda')

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(15.8339, device='cuda:0', grad_fn=<SumBackward0>)